In [77]:
import fastf1 as ff1
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
import time
from fastf1 import plotting

In [1]:
# Limpando o cache **CUIDADO**
#ff1.Cache.clear_cache('./cache')

In [78]:
# Setting up cache for ff1
ff1.Cache.enable_cache('./cache')

In [2]:
RACE_TYPES = {
	'conventional': ['FP1', 'FP2', 'FP3', 'Q', 'R'],
	'sprint_qualifying': ['FP1', 'SQ', 'Sprint', 'Q', 'R'],
}

SESSION_TYPES = [
	'FP1', # Treino livre 1
	'FP2', # TReino livre 2
	'FP3', # Treino livre 3
	'SQ', # Qualificatória da Sprint
	'Sprint', # Sprint...
	'Q', # Qualificatória
	'R', # Corrida
]

In [3]:
SESSION_NAME = 'FP1'
TRACK_NAME = 'Monza'
RACE_YEAR = 2018

In [ ]:
session = ff1.get_session(RACE_YEAR, TRACK_NAME, SESSION_NAME)
session.load()
circuit_info = session.get_circuit_info()
circuit_info

In [ ]:
session.laps

In [ ]:
fastest_lap = session.laps.pick_fastest()
pos = fastest_lap.get_pos_data()
pos

In [9]:
def rotate(xy, *, angle):
	rot_mat = np.array([[np.cos(angle), np.sin(angle)], [-np.sin(angle), np.cos(angle)]])
	return np.matmul(xy, rot_mat)

In [10]:
track = pos.loc[:, ('X', 'Y')].to_numpy()

track_ang = circuit_info.rotation / 180 * np.pi

rotated_track = rotate(track, angle=track_ang)


In [ ]:
marshal_zone = circuit_info.marshal_sectors
marshal_zone

In [ ]:
plt.plot(rotated_track[:, 0], rotated_track[:, 1])
offset_vector = [500, 0]  # offset length is chosen arbitrarily to 'look good'

# Iterate over all corners.
for _, corner in circuit_info.corners.iterrows():
	# Create a string from corner number and letter
	txt = f"{corner['Number']}{corner['Letter']}"

	# Convert the angle from degrees to radian.
	offset_angle = corner['Angle'] / 180 * np.pi

	# Rotate the offset vector so that it points sideways from the track.
	offset_x, offset_y = rotate(offset_vector, angle=offset_angle)

	# Add the offset to the position of the corner
	text_x = corner['X'] + offset_x
	text_y = corner['Y'] + offset_y

	# Rotate the text position equivalently to the rest of the track map
	text_x, text_y = rotate([text_x, text_y], angle=track_ang)

	# Rotate the center of the corner equivalently to the rest of the track map
	track_x, track_y = rotate([corner['X'], corner['Y']], angle=track_ang)

	# Draw a circle next to the track.
	plt.scatter(text_x, text_y, color='grey', s=140)

	# Draw a line from the track to this circle.
	plt.plot([track_x, text_x], [track_y, text_y], color='grey')

	# Finally, print the corner number inside the circle.
	plt.text(text_x, text_y, txt, va='center_baseline', ha='center', size='small', color='white')

for _, zone in marshal_zone.iterrows():
	# Rotate the center of the corner equivalently to the rest of the track map
	track_x, track_y = rotate([zone['X'], zone['Y']], angle=track_ang)

	# Draw a circle next to the track.
	plt.scatter(track_x, track_y, color='red', s=140)

	# Finally, print the corner number inside the circle.
	plt.text(track_x, track_y, zone['Number'], va='center_baseline', ha='center', size='small', color='white')

plt.title(session.event['Location'])
plt.xticks([])
plt.yticks([])
plt.axis('equal')
plt.show()

In [ ]:
# Getting the telemetry data
telemetry = fastest_lap.get_car_data()
# racer = fastest_lap.get_driver()
telemetry

In [ ]:
test = session.weather_data
test

In [ ]:
plt.plot(test['Time'], test['Rainfall']*50, color='blue', label='Rainfall')
plt.plot(test['Time'], test['Humidity'], color='green', label='Humidity')
plt.plot(test['Time'], test['TrackTemp'], color='red', label='Track Temp')
plt.legend()
plt.show()

In [ ]:
drivers = []
for driver in session.drivers:
	drv = session.get_driver(driver)
	drivers.append(drv)

drivers = pd.DataFrame(drivers)
drivers = drivers.drop(['BroadcastName', 'DriverId', 'TeamId', 'FullName', 'HeadshotUrl', 'CountryCode', 'Q1', 'Q2', 'Q3'], axis=True)
drivers

In [ ]:
# Making a bar plot with the drivers
plt.figure(figsize=(10, 5))
plt.bar(drivers['Abbreviation'], drivers['Points'], color=[f"#{color}" for color in drivers['TeamColor']])
plt.title('Drivers Points')
plt.xlabel('Drivers')
plt.ylabel('Points')
plt.show()

In [11]:
START_YEAR_LOG = 2018

registry = {}

In [ ]:
event = ff1.get_event_schedule(START_YEAR_LOG)
event

In [ ]:
sesssion = ff1.get_session(START_YEAR_LOG, 'Monaco', 'R')
sesssion.load()
sesssion

In [ ]:
from datetime import datetime as dt

current_year = dt.now().year
complied_years = []
current_year

In [ ]:


for year in range(START_YEAR_LOG, current_year):
	print(f'Processing year {year}')
	year_data = []
	event = None
	try:
		event = ff1.get_event_schedule(year)
	except Exception as e:
		print(f'Error on {year}')
		print(e)
		continue
	for key, row_value in event.iterrows():
		if not row_value['F1ApiSupport']:
			continue
		if row_value['EventFormat'] not in RACE_TYPES:
			print(f'Event format {row_value["EventFormat"]} not configured.')
			break
		for session_type in RACE_TYPES[row_value['EventFormat']]:
			try:
				session = ff1.get_session(year, row_value['Location'], session_type)
				session.load()
				year_data.append(session)
			except Exception as e:
				print(f'Error on {year} - {row_value["Location"]} - {session_type}')
				print(e)
				break
	registry[year] = year_data
	complied_years.append(year)

In [66]:
class Event:
	def __init__(self, year):
		self.year = year
		self.schedule = ff1.get_event_schedule(year)
		self.num_races = self.schedule['Location'].count()

	def getSchedule(self):
		return self.schedule

	def __str__(self):
		return self.schedule

	def getRace(self, weekend):
		race_week = self.schedule[self.schedule['RoundNumber'] == weekend]
		if race_week is None or race_week.empty:
			race_week = self.schedule[self.schedule['Location'] == weekend]
		if race_week is None or race_week.empty:
			print(f'Weekend {weekend} not found.')
			raise ValueError('Weekend not found')
		print(race_week)
		print(type(race_week))
		if race_week['EventFormat'] not in RACE_TYPES:
			print(f'Event format {race_week["EventFormat"]} not configured.')
			raise TypeError('Event format not configured')
		if self.races[race_week['Location']] and len(self.races[race_week['Location']]) > 0:
			return self.races[race_week['Location']]
		full_race = []
		if not "races" in self.__dict__:
			self.races = {}
		self.races[race_week['Location']] = []
		for session_type in RACE_TYPES[race_week['EventFormat']]:
			try:
				session = ff1.get_session(self.year, weekend, session_type)
				session.load()
				full_race.append(session)
			except Exception as e:
				print(f'Error on {self.year} - {weekend} - {session_type}')
				print(e)
				raise e
		return full_race

	def load_all_races(self):
		print(f'Loading races for {self.year}:')
		races = {}
		for key, row_value in self.schedule.iterrows():
			if not row_value['F1ApiSupport']:
				print(f'No support for {row_value["Location"]}')
				continue
			if row_value['EventFormat'] not in RACE_TYPES:
				print(f'Event format {row_value["EventFormat"]} not configured.')
				break
			races[row_value['Location']] = []
			for session_type in RACE_TYPES[row_value['EventFormat']]:
				try:
					session = ff1.get_session(self.year, row_value['Location'], session_type)
					session.load()
					races[row_value['Location']].append(session)
				except Exception as e:
					print(f'Error on {self.year} - {row_value["Location"]} - {session_type}')
					print(e)
					return False
		self.races = races
		return True

	def getNumRaces(self):
		return self.num_races

	def getRaces(self):
		return self.races

In [67]:
races_event = []

In [ ]:
year_2018 = Event(2018)
# year_2018.load_all_races()
year_2018.getRace('Monza')
# races_event.append(year_2018)
year_2018.getSchedule()

In [ ]:
races_event

In [5]:
import pandas as pd
spa = pd.read_csv('csv/Belgian GP.csv')
spa

,year,session_type,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,2018,FP1,0 days 00:17:55.390000,GAS,10,NaN,1.0,NaN,0 days 00:15:32.833000,0 days 00:17:51.546000,...,True,Toro Rosso,0 days 00:15:32.833000,2018-08-24 09:00:32.851,1,NaN,False,NaN,False,False
1,2018,FP1,0 days 00:37:12.600000,GAS,10,NaN,2.0,2.0,0 days 00:35:17.036000,NaN,...,False,Toro Rosso,0 days 00:17:55.390000,2018-08-24 09:02:55.408,1,NaN,False,NaN,False,False
2,2018,FP1,0 days 00:39:03.255000,GAS,10,0 days 00:01:50.655000,3.0,2.0,NaN,NaN,...,False,Toro Rosso,0 days 00:37:12.600000,2018-08-24 09:22:12.618,1,NaN,False,NaN,False,True
3,2018,FP1,0 days 00:40:52.393000,GAS,10,0 days 00:01:49.138000,4.0,2.0,NaN,NaN,...,False,Toro Rosso,0 days 00:39:03.255000,2018-08-24 09:24:03.273,1,NaN,False,NaN,False,True
4,2018,FP1,0 days 00:42:58.761000,GAS,10,0 days 00:02:06.368000,5.0,2.0,NaN,NaN,...,False,Toro Rosso,0 days 00:40:52.393000,2018-08-24 09:25:52.411,1,NaN,False,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,2018,R,0 days 00:35:54.645000,RAI,7,0 days 00:02:02.327000,7.0,2.0,NaN,0 days 00:35:51.921000,...,True,Ferrari,0 days 00:33:52.318000,2018-08-26 13:28:52.344000,1,16.0,False,NaN,False,False
2165,2018,R,0 days 00:38:27.590000,RAI,7,NaN,8.0,3.0,0 days 00:36:26.391000,0 days 00:38:23.994000,...,False,Ferrari,0 days 00:35:54.645000,2018-08-26 13:30:54.671000,1,16.0,False,NaN,False,False
2166,2018,R,0 days 00:21:03.949000,LEC,16,NaN,1.0,1.0,NaN,NaN,...,True,Sauber,0 days 00:18:34.162000,2018-08-26 13:13:34.188000,124,NaN,False,NaN,True,False
2167,2018,R,0 days 00:21:03.949000,ALO,14,NaN,1.0,1.0,NaN,NaN,...,True,McLaren,0 days 00:18:34.162000,2018-08-26 13:13:34.188000,124,NaN,False,NaN,True,False


In [6]:
spa[spa['year'] == 2019]

,year,session_type,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
